In [ ]:
pip install nltk

In [43]:
import nltk
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('omw-1.4')


from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()
import json
import pickle
import random
import numpy as np
from tensorflow.keras.models import load_model
model = load_model('/content/drive/MyDrive/ChatBOT/chatbot_model_(1).h5')
intents = json.loads(open('/content/drive/MyDrive/ChatBOT/intents.json', encoding="utf8").read())
words = pickle.load(open('/content/drive/MyDrive/ChatBOT/words.pkl','rb'))
classes = pickle.load(open('/content/drive/MyDrive/ChatBOT/classes.pkl','rb'))

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


In [ ]:
def clean_up_sentence(sentence):
    #tokenize the pattern-split words into array
    sentence_words = nltk.word_tokenize(sentence)
    #stem each word-create short form for word
    sentence_words = [lemmatizer.lemmatize(word.lower()) for word in sentence_words]
    print("sentence words",sentence_words)
    return sentence_words

In [ ]:
def bow(sentence, words, show_details=True):
    #tokenize the pattern
    sentence_words = clean_up_sentence(sentence)
    #bag of words-matrix of N words
    bag = [0]*len(words)

    for s in sentence_words:
        for i,w in enumerate(words):
            if w == s:
                #assign 1 if current word is in the vocabulary position
                bag[i] =1
                if show_details:
                    print("found in bag: %s" % w)
    return(np.array(bag))

In [ ]:
def predict_class(sentence, model):
    # filter out predictions below a threshold
    p = bow(sentence, words,show_details=False)
    print("p=",p)
    res = model.predict(np.array([p]))[0]
    print("res=",res)
    ERROR_THRESHOLD = 0.25
    results = [[i,r] for i,r in enumerate(res) if r>ERROR_THRESHOLD]
    print("result=",results)
    #sort by strength of probability
    results.sort(key=lambda x: x[1], reverse=True)
    return_list = []
    for r in results:
        return_list.append({"intent": classes[r[0]], "probability": str(r[1])})

    return return_list if return_list else [{"intent": "no_match", "probability": "0"}]

In [ ]:
import random
def getResponse(ints, intents_json):
    tag = ints[0]['intent']
    if tag == "no_match":
        return "I'm sorry, I don't understand that."
    print("tag=",tag)
    list_of_intents = intents_json['intents']
    print("list of intents=",list_of_intents)
    for i in list_of_intents:
      if(i['tag']== tag):
          result = random.choice(i['responses'])
          print("result=",result)
          break
    return result

In [ ]:
def chatbot_response(msg):
    ints = predict_class(msg, model)
    print("ints=",ints)
    res = getResponse(ints, intents)
    print("chatbot response=",res)
    return res
#

In [ ]:
!pip install pyngrok

In [ ]:
from flask import Flask
from pyngrok import ngrok
ngrok_key = "2maxy6o104eSyuT2jAD7bvSJf7N_5S5dGVTXhuM8UP2mirrsp"
port = 5000

In [ ]:
!pip install flask-cors

In [44]:
#from flask_ngrok import run_with_ngrok
import logging
from flask import Flask,jsonify
from flask_cors import CORS
app = Flask(__name__)
ngrok.set_auth_token(ngrok_key)
public_url=ngrok.connect(port).public_url
CORS(app)
#run_with_ngrok(app)
@app.route("/",methods=['GET'])
def hello():
  return jsonify({'message':'server is on'})


def decrypt(msg):
  #i/p=what+is+maching+learning
  #o/p= what is machine learning
  #remove + and replace it with spaces
  string=msg
  new_string=string.replace("+"," ")
  return new_string

@app.route("/query/<sentence>")
def query_chatbot(sentence):
  #decrypt
  dec_msg=decrypt(sentence)

  response=chatbot_response(dec_msg)
  json_obj=jsonify({'top':{"res":response}})
  return json_obj



print(f"to access link click {public_url}")

app.run(port = port)



to access link click https://4e59-34-148-29-48.ngrok-free.app
 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit
INFO:werkzeug:127.0.0.1 - - [26/Sep/2024 05:49:36] "GET / HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [26/Sep/2024 05:49:37] "GET /favicon.ico HTTP/1.1" 404 -
INFO:werkzeug:127.0.0.1 - - [26/Sep/2024 05:50:02] "GET /query.hello HTTP/1.1" 404 -


sentence words ['hello']
p= [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0

INFO:werkzeug:127.0.0.1 - - [26/Sep/2024 05:50:09] "GET /query/hello HTTP/1.1" 200 -


res= [1.41016506e-02 8.83831060e-04 1.03698100e-03 1.03505049e-03
 4.82959644e-04 1.53969871e-04 1.32073485e-03 2.06546183e-03
 9.62448539e-04 1.18469656e-03 6.68039487e-04 1.63258679e-04
 3.93763708e-04 8.73112353e-04 2.25679646e-03 5.42976835e-04
 4.66930168e-03 5.72081655e-04 3.20662919e-04 9.44901185e-05
 6.02758664e-04 1.44675805e-03 3.41519073e-04 1.68004085e-03
 5.73040498e-03 5.80635853e-04 2.80575734e-02 1.87503407e-04
 9.53452138e-04 1.59837818e-03 2.76341848e-03 6.38996530e-03
 1.07761740e-03 2.86943140e-03 2.56482884e-03 1.54607068e-03
 1.31854014e-02 1.70663977e-03 2.80084874e-04 8.23970404e-05
 7.54034554e-04 3.29676666e-03 9.00893996e-04 1.84389530e-04
 4.23117279e-04 6.56052213e-03 6.15605823e-05 8.00328562e-04
 8.77474260e-04 4.12049267e-04 1.46335355e-04 8.64473986e-04
 1.91910670e-03 1.02043303e-03 1.34704611e-03 5.93596080e-04
 7.04554433e-04 4.81451768e-03 3.76803073e-04 7.30837055e-04
 3.50461225e-04 1.33863368e-05 5.92813420e-04 6.49321373e-05
 2.54637073e-03 1.3

...

res= [5.2282275e-03 7.1252277e-04 4.4054753e-04 1.1986704e-03 2.3132425e-03
 1.9781373e-04 5.6929472e-03 7.9973917e-03 5.3986823e-03 2.8567656e-03
 4.1980385e-03 9.3008020e-05 3.2335881e-03 6.1890353e-03 2.6131780e-03
 5.6992687e-04 2.4519206e-03 1.1205330e-03 1.0784794e-04 9.9443987e-06
 1.0355026e-03 3.6853678e-05 4.8802485e-04 4.6112631e-03 5.6195720e-03
 1.1507160e-03 9.6595613e-04 3.3171225e-04 1.3547270e-04 8.9673912e-03
 8.9720143e-03 3.4265961e-02 9.3130238e-04 8.8381357e-03 7.4256840e-04
 5.8392657e-04 3.0483145e-04 1.7130798e-04 1.0424892e-03 9.7092363e-04
 5.8945396e-04 1.5433093e-03 4.4955458e-03 1.4152995e-04 1.0715715e-03
 2.0592064e-02 2.7054464e-04 1.0793447e-04 1.7016960e-03 8.7261747e-04
 3.7239130e-05 7.0138052e-03 5.2584452e-04 2.7489057e-03 9.5427502e-04
 1.1500473e-03 2.1720077e-04 9.8893289e-03 1.4906637e-03 5.2820600e-04
 2.8680433e-03 3.6674326e-06 5.9394084e-04 5.4322776e-05 5.7516256e-03
 1.0187213e-03 2.8057976e-04 1.8270643e-03 6.2272133e-04 1.0919542e-03
 

INFO:werkzeug:127.0.0.1 - - [26/Sep/2024 06:07:33] "GET /query/bye HTTP/1.1" 200 -


res= [1.46769287e-06 1.10583323e-06 4.11738093e-08 1.77705874e-06
 1.21979485e-06 6.17234406e-08 2.99996100e-06 2.27806413e-06
 1.00339889e-06 8.69086398e-06 4.96623943e-06 1.73997228e-06
 1.51233480e-05 1.46097236e-03 1.12327109e-06 3.39935582e-08
 6.11294399e-06 6.69263045e-07 1.64156120e-07 4.99777064e-09
 1.57513540e-07 8.32962641e-08 8.09218861e-08 5.97652502e-07
 1.09109315e-04 1.32855433e-07 4.64580580e-06 3.93091057e-07
 3.21466786e-07 3.00213202e-07 9.68229870e-05 1.66067475e-04
 3.04862340e-07 6.43923631e-05 2.69643988e-05 1.76477215e-06
 3.34117358e-05 5.78840513e-07 4.96361849e-07 3.02544720e-07
 1.59646584e-06 1.74074485e-05 5.77960463e-06 1.04029768e-03
 1.12272438e-07 2.57769607e-05 5.64908873e-07 2.14800039e-05
 1.27223902e-05 2.19464980e-07 4.29559588e-09 2.61609125e-06
 2.83832009e-07 2.09324116e-05 8.95904790e-08 1.00085435e-05
 6.08666994e-07 1.22692683e-04 3.92106776e-05 2.84973212e-04
 2.57550673e-05 2.15131184e-08 1.39760632e-05 3.55487835e-08
 1.43018624e-05 5.3

INFO:werkzeug:127.0.0.1 - - [26/Sep/2024 06:07:56] "GET /query/how+can+you+help+me HTTP/1.1" 200 -


res= [6.52358631e-06 5.48468915e-06 5.00383237e-07 4.11385599e-06
 1.15821717e-06 5.05321680e-07 2.12412142e-05 4.12810477e-05
 4.82521755e-05 5.58252214e-06 6.28150531e-07 1.51178174e-04
 5.18322813e-05 1.55651742e-05 1.55651821e-06 1.23444110e-07
 2.80572476e-06 2.43316640e-06 9.90660101e-06 1.79346207e-05
 1.72178670e-06 3.45145745e-05 2.01093098e-06 9.59773729e-07
 7.87047466e-05 1.31625058e-07 6.57273317e-03 2.61827745e-06
 4.92069594e-05 6.56191987e-05 1.24146582e-05 1.33194408e-05
 8.47030492e-07 5.72900808e-06 2.39362828e-02 5.66425570e-06
 1.34440470e-05 6.41911299e-07 1.56922069e-05 4.63119686e-05
 1.01024725e-05 7.27358682e-04 8.58217663e-06 1.90126570e-06
 7.22616687e-06 2.77042964e-05 1.57572083e-06 2.89444288e-04
 8.25297832e-07 6.44999773e-07 1.50817669e-07 5.81042343e-07
 1.33710302e-04 9.13661665e-07 2.65149765e-07 1.04356080e-03
 7.97224220e-06 2.28118165e-06 3.39669015e-07 6.06444416e-07
 5.37530104e-05 1.38831208e-04 2.45539763e-06 2.00996465e-06
 9.90860462e-06 1.8